### 해커톤
- 해커톤에서 구현한 기능에 대한 고찰과 분석
- 반복되었던 오류에 대한 원인 분석
- 코드 간결화
- 추가적으로 구현할 수 있는 부분에 대한 분석


- 구현한 파트: account 파트
    - 회원가입(signup) 구현
    - 로그인(login) 구현
    - 로그아웃(logout) 구현

### accounts -> models.py

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractUser


class CustomUser(AbstractUser):
    username = models.CharField(max_length=50, unique=True)
    password = models.CharField(max_length=50)
    password1 = models.CharField(max_length=50)
    phone_num = models.CharField(max_length=11)
    user_address = models.CharField(max_length=200, blank = True, null = True)
    user_bool = models.BooleanField(default=False)

### unique = True
- 오류가 반복된 부분
- 계속하여 username이 중복된다는 오류가 계속 발생함
    - username 필드를 고유한 값으로 만들어줬어야함. 
- 'unique = True' -> 고유한 값으로 하는 필드에 적용

### 두개 이상의 필드를 고유화 할 경우

In [ ]:
class CustomUser(AbstractUser):
    class Meta:
        constraints = [
            models.UniqueConstraint(
            fields = ["username","email"])
        ]

### password를 입력하라는 오류

In [ ]:
# 기존 CustomUser
class CustomUser(AbstractUser):
    password1 = models.CharField(max_length=50)
    password2 = models.CharField(max_length=50)

### 오류 원인에 대한 분석
- AbstractUser을 상속받았기 때문에 password 값이 없어서는 안되는것인지..??
- AbstractBaseUSer의 경우는 password를 갖고 있지만 AbstractUser에는 명시되어 있지 않음

### AbstractBaseUser vs AbstractUser
- AbstractBaseUser
    - password와 last_login만 던져줌
    - 세밀한 커스텀 유저를 만들 때 사용 -> ex) 페이스북 로그인등을 통한 로그인
- AbstractUser
    - 많은 값을 던져줌
    - settings.py에 AUTH_USER_MODEL='모듈.클래스이름'을 추가해줘야함

### accounts -> serializers.py

In [ ]:
from rest_framework import serializers
from .models import CustomUser

class LoginSerializer(serializers.Serializer):
    username = serializers.CharField(max_length=150)
    password = serializers.CharField(max_length=128)

class SignupSerializer(serializers.ModelSerializer):
    class Meta:
        model = CustomUser
        fields = ['username','password','password1','phone_num','user_address','user_bool']

### accounts -> views.py

In [ ]:
from .models import CustomUser
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status
from .serializers import LoginSerializer, SignupSerializer
from django.contrib import auth
from django.contrib.auth.hashers import make_password

# Create your views here.

@api_view(['POST'])
def signup(request):
    serializer = SignupSerializer(data=request.data)
    if serializer.is_valid():
        if len(serializer.validated_data['password']) >= 8 and any(i.isalpha() for i in serializer.validated_data['password']) and any(i.isdigit() for i in serializer.validated_data['password']):   
            if serializer.validated_data['password'] == serializer.validated_data['password1']:
                new_user = serializer.save(password = make_password(serializer.validated_data['password']))
                auth.login(request, new_user)
                return Response(status=status.HTTP_200_OK)
    return Response(status=status.HTTP_400_BAD_REQUEST)

'''
로그인
'''
@api_view(['POST'])
def login(request):
    serializer = LoginSerializer(data=request.data)
    if serializer.is_valid():
        user = auth.authenticate(
            request = request, 
            username = serializer.data['username'],
            password = serializer.data['password']
        )
        if user is not None:
            auth.login(request, user)
            return Response(status=status.HTTP_200_OK)
        return Response(status=status.HTTP_404_NOT_FOUND)
    return Response(status=status.HTTP_400_BAD_REQUEST)

'''
로그아웃
'''
@api_view(['POST'])
def logout(request):
    auth.logout(request)
    return Response(status=status.HTTP_200_OK)

### signup 함수
- isdigit(), isalpha() 모두 문자열에 대해 적용 가능 -> 따로 예외처리 구문 필요 없음
- serializer.validated_data => 유효성 검사를 한 데이터에 대한 처리
    - is.valid() 이후 유효성 검사를 통과한 데이터이기 때문
    - serializer.password가 아닌 serializer.validated_data['password']로 접근